<a href="https://colab.research.google.com/github/jyryu3161/SyntheticBiology/blob/main/lec5_metabolic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cobra

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 31.6 MB/s eta 0:00:00


In [2]:
"""
cobrapy_toy_tutorial.py
A minimal illustrative tutorial script that builds a very small toy metabolic model with cobrapy and walks through the key inspection steps you asked for.

Run it with:
    python cobrapy_toy_tutorial.py

Prerequisites:
    pip install cobra
"""

import cobra
from cobra import Model, Reaction, Metabolite


def build_toy_model() -> Model:
    """Construct and return a tiny three‑reaction toy model."""
    model = Model("toy")

    # --- Metabolites -------------------------------------------------------
    A = Metabolite("A_c", formula="C", name="Metabolite A", compartment="c")
    B = Metabolite("B_c", formula="C", name="Metabolite B", compartment="c")
    C = Metabolite("C_c", formula="C", name="Metabolite C", compartment="c")

    # --- Reactions ---------------------------------------------------------
    R1 = Reaction("R1")
    R1.name = "A → B"
    R1.lower_bound, R1.upper_bound = 0, 1000  # irreversible
    R1.add_metabolites({A: -1, B: 1})

    R2 = Reaction("R2")
    R2.name = "B → C"
    R2.lower_bound, R2.upper_bound = 0, 1000
    R2.add_metabolites({B: -1, C: 1})

    biomass = Reaction("Biomass")
    biomass.name = "Biomass synthesis"
    biomass.lower_bound, biomass.upper_bound = 0, 1000
    biomass.add_metabolites({C: -1})  # drains C to make biomass

    model.add_reactions([R1, R2, biomass])
    model.objective = biomass
    return model


def main():
    model = build_toy_model()

    # 1. Metabolite 정보 확인 -------------------------------------------------
    print("=== Metabolites ===")
    for m in model.metabolites:
        # id\tname\tformula\tcompartment
        print(f"{m.id}\t{m.name}\t{m.formula}\t{m.compartment}")

    # 2. Reaction 정보 확인 ---------------------------------------------------
    print("\n=== Reactions ===")
    for r in model.reactions:
        # id\treaction string\tbounds
        print(f"{r.id}\t{r.reaction}\t[{r.lower_bound}, {r.upper_bound}]")

    # 3. Compartment 정보 확인 ----------------------------------------------
    print("\n=== Compartments ===")
    for cid, desc in model.compartments.items():
        print(f"{cid}: {desc}")

    # 4. Toy model 전체 reaction summary ------------------------------------
    print("\n=== Model Summary ===")
    print(model.summary())

    # (선택) built‑in textbook model 빠르게 보기 -----------------------------
    # textbook = cobra.test.create_test_model("textbook")
    # print("\nTextbook model summary:")
    # print(textbook.summary())


if __name__ == "__main__":
    main()


=== Metabolites ===
A_c	Metabolite A	C	c
B_c	Metabolite B	C	c
C_c	Metabolite C	C	c

=== Reactions ===
R1	A_c --> B_c	[0, 1000]
R2	B_c --> C_c	[0, 1000]
Biomass	C_c --> 	[0, 1000]

=== Compartments ===
c: 

=== Model Summary ===
Objective
1.0 Biomass = 0.0

Uptake
------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []

Secretion
---------
Empty DataFrame
Columns: [Metabolite, Reaction, Flux, C-Number, C-Flux]
Index: []


=== FBA Solution ===
<Solution 0.000 at 0x7b3022b62bd0>

Flux Distribution:
R1	0.0
R2	0.0
Biomass	0.0
